# Question Answering Model 
## no trainer

- dataset
- torch
- transformers
- transformers[torch]
- evaluate

import packages

In [1]:
from datasets import load_dataset, disable_caching
from transformers import (
    AutoTokenizer,
    default_data_collator,
    get_scheduler,
    AutoModelForQuestionAnswering
)
from torch.utils.data import DataLoader
from torch.optim import AdamW
import torch
from torch.cuda import amp

from collections import defaultdict, Counter
from tqdm.auto import tqdm
import numpy as np

# for batch_bald_query
import evaluate
from scipy import stats
from sklearn.metrics import pairwise_distances
import pdb

import os
import re
import string

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


arguments.py

In [2]:
STRATEGY_NAME = 'BatchBALDDropout'
DATA_NAME = 'BioASQ'
EXPE_ROUND = 3
MODEL_BATCH = 8
MAX_LENGTH = None
LEARNING_RATE = 3e-5
MODEL_NAME = 'RoBERTa'
LOW_RES = True
NUM_TRAIN_EPOCH = 3
UNIQ_CONTEXT = False
if LOW_RES:
    args_input_quota = 200
    NUM_QUERY = 50
else:
    NUM_INIT_LB = 500 # 1000
    args_input_quota = 2000 # 200
    NUM_QUERY = 500 # 50
ITERATION = int(args_input_quota / NUM_QUERY)

stride = 128

model_dir = '/mount/arbeitsdaten31/studenten1/linku/dev_models'
CACHE_DIR = '/mount/arbeitsdaten31/studenten1/linku/linku/.cache'

preprocess.py

In [3]:
def preprocess_training_examples(examples, tokenizer):
    # no ['offset_mapping'], for .train() and .eval()
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


def preprocess_training_features(examples, tokenizer):
    # keep ["offset_mapping"] and ["example_id"], for compute_metrics()
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs["offset_mapping"]
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []
    example_ids = []
    contexts = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)
        
        # added for used in unlabel data predict
        example_ids.append(examples["id"][sample_idx]) 
        # added for unique context filter
        contexts.append(examples["context"][sample_idx])

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["context"] = contexts
    inputs["example_id"] = example_ids
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

def preprocess_validation_examples(examples, tokenizer):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

def preprocess_training_examples_lowRes(examples, tokenizer):
    # no ['offset_mapping'], for .train() and .eval()
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    detected_answers = examples["detected_answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = detected_answers[sample_idx]
        start_char = answer["char_spans"][0]["start"][0]
        end_char = answer["char_spans"][0]["end"][0]
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

def preprocess_training_features_lowRes(examples, tokenizer):
    # keep ["offset_mapping"] and ["example_id"], for compute_metrics()
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs["offset_mapping"]
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["detected_answers"]
    start_positions = []
    end_positions = []
    example_ids = []
    contexts = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["char_spans"][0]["start"][0]
        end_char = answer["char_spans"][0]["end"][0]
        sequence_ids = inputs.sequence_ids(i)
        
        # added for used in unlabel data predict
        example_ids.append(examples["qid"][sample_idx])
        # added for unique context filter
        contexts.append(examples['context'][sample_idx])

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["context"] = contexts
    inputs["example_id"] = example_ids
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

def preprocess_validation_examples_lowRes(examples, tokenizer):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["qid"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs


utils.py

In [4]:
def get_aubc(quota, bsize, resseq):
	# it is equal to use np.trapz for calculation
	ressum = 0.0
	if quota % bsize == 0:
		for i in range(len(resseq)-1):
			ressum = ressum + (resseq[i+1] + resseq[i]) * bsize / 2

	else:
		for i in range(len(resseq)-2):
			ressum = ressum + (resseq[i+1] + resseq[i]) * bsize / 2
		k = quota % bsize
		ressum = ressum + ((resseq[-1] + resseq[-2]) * k / 2)
	ressum = round(ressum / quota,3)
	
	return ressum


def get_mean_stddev(datax):
	return round(np.mean(datax),4),round(np.std(datax),4)


def get_unlabel_data(n_pool, labeled_idxs, train_dataset):
    unlabeled_idxs = np.arange(n_pool)[~labeled_idxs]
    unlabeled_data = train_dataset.select(indices=unlabeled_idxs)
    return unlabeled_idxs, unlabeled_data


def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)


def get_model(m):
	if m.lower() == 'bert':
		return 'bert-base-uncased'
	elif m.lower() == 'bertlarge':
		return 'bert-large-uncased'
	elif m.lower() == 'roberta':
		return 'roberta-base'
	elif m.lower() == 'robertalarge':
		return 'roberta-large'


def get_context_id(data):
    context_id = {}
    for i, c in enumerate(set(data['context'])):
        context_id[c] = i+1
    return context_id


def preprocess_data(train_data, val_data):
	tokenizer = AutoTokenizer.from_pretrained(get_model(MODEL_NAME))

	if LOW_RES:
		train_dataset = train_data.map(
			preprocess_training_examples_lowRes,
			batched=True,
			remove_columns=train_data.column_names,
			fn_kwargs=dict(tokenizer=tokenizer)
		)
		train_features = train_data.map(
			preprocess_training_features_lowRes,
			batched=True,
			remove_columns=train_data.column_names,
			fn_kwargs=dict(tokenizer=tokenizer)
		)
		val_dataset = val_data.map(
			preprocess_validation_examples_lowRes,
			batched=True,
			remove_columns=val_data.column_names,
			fn_kwargs=dict(tokenizer=tokenizer)
		)
		val_features = val_data.map(
			preprocess_validation_examples_lowRes,
			batched=True,
			remove_columns=val_data.column_names,
			fn_kwargs=dict(tokenizer=tokenizer)
		)
	else:
		train_dataset = train_data.map(
			preprocess_training_examples,
			batched=True,
			remove_columns=train_data.column_names,
			fn_kwargs=dict(tokenizer=tokenizer)
		)
		train_features = train_data.map(
			preprocess_training_features,
			batched=True,
			remove_columns=train_data.column_names,
			fn_kwargs=dict(tokenizer=tokenizer)
		)
		val_dataset = val_data.map(
			preprocess_validation_examples,
			batched=True,
			remove_columns=val_data.column_names,
			fn_kwargs=dict(tokenizer=tokenizer)
		)
		val_features = val_data.map(
			preprocess_validation_examples,
			batched=True,
			remove_columns=val_data.column_names,
			fn_kwargs=dict(tokenizer=tokenizer)
		)

	train_dataset.set_format("torch")
	train_features.set_format("torch")
	val_dataset = val_dataset.remove_columns(["offset_mapping"])
	val_dataset.set_format("torch")
	val_features.set_format("torch")

	return train_dataset, train_features, val_dataset, val_features


def load_dataset_mrqa(d):
	'''
	return train_set, val_set
	'''
	data = load_dataset("mrqa", cache_dir=CACHE_DIR)
	if d == 'squad':
		# the first to 86588th in train set
		# the first to 10507th in val set
		squad_train = data['train'].select(range(86588))
		squad_val = data['validation'].select(range(10507))
		for t in squad_train: assert t['subset'] == 'SQuAD', 'Please select corrrect train data for SQuAD.'
		for v in squad_val: assert v['subset'] == 'SQuAD', 'Please select corrrect validation data for SQuAD.'
		return squad_train, squad_val
	elif d == 'newsqa':
		# the 86589th to 160748th in train set
		# the 10508th to 14719th in val set
		data_set = data['train'].select(range(86588, 160748))
		newsqa_train = data_set.shuffle(1127).select(range(10000))
		newsqa_val = data['validation'].select(range(10507, 14719))
		for t in newsqa_train: assert t['subset'] == 'NewsQA', 'Please select corrrect train data for NewQA.'
		for v in newsqa_val: assert v['subset'] == 'NewsQA', 'Please select corrrect validation data for NewQA.'
		return newsqa_train, newsqa_val
	elif d == 'searchqa':
		# the 222437th to 339820th in train set
		# the 22505th to 39484th in val set
		data_set = data['train'].select(range(222436, 339820))
		searchqa_train = data_set.shuffle(1127).select(range(10000))
		searchqa_val = data['validation'].select(range(22504, 39484))	
		for t in searchqa_train: assert t['subset'] == 'SearchQA', 'Please select corrrect train data for SearchQA.'
		for v in searchqa_val: assert v['subset'] == 'SearchQA', 'Please select corrrect validation data for SearchQA.'
		return searchqa_train, searchqa_val
	elif d == 'bioasq':
		# the first to the 1504th in the test set
		sub = data['test'].select(range(1504))
		len_sub_val = len(sub) // 10
		bioasq_train = sub.select(range(len_sub_val, len(sub)))
		bioasq_val = sub.select(range(len_sub_val))
		for t in bioasq_train: assert t['subset'] == 'BioASQ', 'Please select corrrect train data for BioASQ.'
		for v in bioasq_val: assert v['subset'] == 'BioASQ', 'Please select corrrect validation data for BioASQ.'
		return bioasq_train, bioasq_val
	elif d == 'textbookqa':
		# the 8131st to 9633rd
		sub = data['test'].select(range(8130, 9633))
		len_sub_val = len(sub) // 10
		textbookqa_train = sub.select(range(len_sub_val, len(sub)))
		textbookqa_val = sub.select(range(len_sub_val)) 
		for t in textbookqa_train: assert t['subset'] == 'TextbookQA', 'Please select corrrect train data for TextbookQA.'
		for v in textbookqa_val: assert v['subset'] == 'TextbookQA', 'Please select corrrect validation data for TextbookQA.'
		return textbookqa_train, textbookqa_val
	elif d == 'drop': # Discrete Reasoning Over Paragraphs
		# the 1505th to 3007th in test set
		sub = data['test'].select(range(1504, 3007))
		len_sub_val = len(sub) // 10
		drop_train = sub.select(range(len_sub_val, len(sub)))
		drop_val = sub.select(range(len_sub_val))
		for t in drop_train: assert t['subset'] == 'DROP', 'Please select corrrect train data for DROP.'
		for v in drop_val: assert v['subset'] == 'DROP', 'Please select corrrect validation data for DROP.'
		return drop_train, drop_val
	

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def evaluation(theoretical_answers, predicted_answers, skip_no_answer=False):
    '''
	theoretical_answers, datatype=dict
	{strings of id: list of ground truth answers}
	predicted_answers, datatype=dict
	{strings of id: strings of prediction text}
	'''
    f1 = exact_match = total = 0
    for qid, ground_truths in theoretical_answers.items():
        if qid not in predicted_answers:
            if not skip_no_answer:
                message = 'Unanswered question %s will receive score 0.' % qid
                print(message)
                total += 1
            continue
        total += 1
        prediction = predicted_answers[qid]
        exact_match += metric_max_over_ground_truths(
            exact_match_score, prediction, ground_truths)
        f1 += metric_max_over_ground_truths(
            f1_score, prediction, ground_truths)

    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1}


def save_model(device, pretrain_dir, strategy_dir):
    '''
    Copy and save model from pretrain_models to current trained models.
    '''
    pretrain_model = AutoModelForQuestionAnswering.from_pretrained(pretrain_dir).to(device)
    model_to_save = pretrain_model.module if hasattr(pretrain_model, 'module') else pretrain_model 
    model_to_save.save_pretrained(strategy_dir)


def init_centers(X, K):
    ind = np.argmax([np.linalg.norm(s, 2) for s in X])
    mu = [X[ind]]
    indsAll = [ind]
    centInds = [0.] * len(X)
    cent = 0
    print('#Samps\tTotal Distance')
    while len(mu) < K:
        if len(mu) == 1:
            D2 = pairwise_distances(X, mu).ravel().astype(float)
        else:
            newD = pairwise_distances(X, [mu[-1]]).ravel().astype(float)
            for i in range(len(X)):
                if D2[i] >  newD[i]:
                    centInds[i] = cent
                    D2[i] = newD[i]
        print(str(len(mu)) + '\t' + str(sum(D2)), flush=True)
        if sum(D2) == 0.0: pdb.set_trace()
        D2 = D2.ravel().astype(float)
        Ddist = (D2 ** 2)/ sum(D2 ** 2)
        customDist = stats.rv_discrete(name='custm', values=(np.arange(len(D2)), Ddist))
        ind = customDist.rvs(size=1)[0]
        while ind in indsAll: ind = customDist.rvs(size=1)[0]
        mu.append(X[ind])
        indsAll.append(ind)
        cent += 1
    return indsAll

def get_unique_sample(labeled_idxs, q_idxs, n_pool, train_features, iteration=0):
	if LOW_RES:
		num_query_i = NUM_QUERY * iteration
		print('num_query_i in get_unique_sample in LOW_RES:', num_query_i)
	else:
		num_query_i = NUM_QUERY * iteration + NUM_INIT_LB
		print('num_query_i in get_unique_sample:', num_query_i)

	difference_i = 0
	num_set_ex_id_i = 0

	n = 0

	while num_set_ex_id_i < num_query_i:
		labeled_idxs[q_idxs[:NUM_QUERY + difference_i]] = True	# get first num_query, e.g. 50
		iter_i_labeled_idxs = np.arange(n_pool)[labeled_idxs]
		print('len(iter_i_labeled_idxs):', len(iter_i_labeled_idxs))

		iter_i_samples = train_features.select(indices=iter_i_labeled_idxs)
		num_set_ex_id_i = len(set(iter_i_samples['example_id']))
		print('number of unique example id:', num_set_ex_id_i)

		assert num_set_ex_id_i <= num_query_i, 'Select too many examples!'
		assert num_set_ex_id_i > 0, "Did not select examples!"

		difference_i = num_query_i - num_set_ex_id_i
		print('difference_i', difference_i)

		n += 1
		if n == 3: break
	
	return iter_i_labeled_idxs

### load dataset

In [5]:
disable_caching()
if LOW_RES:
	## set dir
	pretrain_model_dir = '/mount/arbeitsdaten31/studenten1/linku/pretrain_models' + '/' + MODEL_NAME + '_SQuAD_full_dataset_lr_3e-5'
	strategy_model_dir = model_dir + '/lowRes_' + str(args_input_quota) + '_' + STRATEGY_NAME + '_' + MODEL_NAME +  '_' + DATA_NAME
	## load data
	train_data, val_data = load_dataset_mrqa(DATA_NAME.lower())
else:
	## set dir
	strategy_model_dir = model_dir + '/' + str(NUM_INIT_LB) + '_' + str(args_input_quota) + '_' + STRATEGY_NAME + '_' + MODEL_NAME +  '_' + DATA_NAME
	## load data
	squad = load_dataset(DATA_NAME.lower(), cache_dir=CACHE_DIR)
	train_data = squad["train"]
	val_data = squad["validation"]
	print('Use full training data and full testing data.')

Found cached dataset mrqa (/mount/arbeitsdaten31/studenten1/linku/linku/.cache/mrqa/plain_text/1.1.0/1f2cf5ac32b43f864e6f91d384057a16b69b7d13ba9bcaa200ac277c90938d19)


  0%|          | 0/3 [00:00<?, ?it/s]

model.py

In [363]:
def to_train(num_train_epochs, train_dataloader, device, model, optimizer, lr_scheduler, record_loss=False):
	if LOW_RES:
		print('Training was performed using {} query data, i.e. {} data.'.format(NUM_QUERY, len(train_dataloader.dataset)))
	else:
		print('Training was performed using the sum of {} initial data and {} query data, i.e. {} data.'.format(NUM_INIT_LB, NUM_QUERY, len(train_dataloader.dataset)))
	
	for epoch in range(num_train_epochs):
		model.train()
		for step, batch in enumerate(tqdm(train_dataloader, desc="Training")):
			batch = {key: value.to(device) for key, value in batch.items()}
			outputs = model(**batch)
			loss = outputs.loss
			loss.backward()

			optimizer.step()
			lr_scheduler.step()
			optimizer.zero_grad()

		if record_loss:
			print('Train Epoch: {}\tLoss: {:.6f}'.format(epoch, loss.item()))

	model_to_save = model.module if hasattr(model, 'module') else model 
	model_to_save.save_pretrained(strategy_model_dir)
	print('TRAIN done!')

metric = evaluate.load("squad")

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = defaultdict(list)
    max_answer_length = 30
    n_best = 20
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples, desc="Computing metrics"):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

def compute_metrics_lowRes(start_logits, end_logits, features, examples):
    example_to_features = defaultdict(list)
    max_answer_length = 30
    n_best = 20
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = dict()
    for example in tqdm(examples, desc="Computing metrics"):
        example_id = example["qid"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers[example_id] = best_answer["text"]
        else:
            predicted_answers[example_id] = ""

    theoretical_answers = dict()
    for ex in examples: theoretical_answers[ex["qid"]] = ex["answers"]
    return evaluation(theoretical_answers, predicted_answers)

def get_pred(dataloader, device, features, examples):
    model = AutoModelForQuestionAnswering.from_pretrained(strategy_model_dir).to(device)
    
    model.eval()
    start_logits = []
    end_logits = []

    for batch in tqdm(dataloader, desc="Evaluating_pred"):
        batch = {key: value.to(device) for key, value in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(outputs.start_logits.cpu().numpy())
        end_logits.append(outputs.end_logits.cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(features)]
    end_logits = end_logits[: len(features)]

    if LOW_RES:
        return compute_metrics_lowRes(start_logits, end_logits, features, examples)
    else:
        return compute_metrics(start_logits, end_logits, features, examples)

def get_prob_dropout_split(dataloader, device, features, examples, n_drop=10):
    ## use tensor to save the answers

    model = AutoModelForQuestionAnswering.from_pretrained(strategy_model_dir).to(device)
    
    model.train()

    probs = torch.zeros([n_drop, len(dataloader.dataset), 200])
    
    for i in range(n_drop):
        start_logits = []
        end_logits = []
        for batch in tqdm(dataloader, desc="Evaluating_prob_dropout"):
            batch = {key: value.to(device) for key, value in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)

            start_logits.append(outputs.start_logits.cpu().numpy())
            end_logits.append(outputs.end_logits.cpu().numpy())

        start_logits = np.concatenate(start_logits)
        end_logits = np.concatenate(end_logits)
        start_logits = start_logits[: len(features)]
        end_logits = end_logits[: len(features)]

        example_to_features = defaultdict(list)
        max_answer_length = 30
        n_best = 20
            
        for idx, feature in enumerate(features):
            example_to_features[feature["example_id"]].append(idx)

        for example in tqdm(examples, desc="Computing metrics"):
            if LOW_RES:
                example_id = example["qid"]
            else:
                example_id = example["id"]
            answers = []

            # Loop through all features associated with that example
            for feature_index in example_to_features[example_id]:
                start_logit = start_logits[feature_index]
                end_logit = end_logits[feature_index]
                offsets = features[feature_index]["offset_mapping"]

                start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
                end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
                for start_index in start_indexes:
                    for end_index in end_indexes:
                        # Skip answers that are not fully in the context
                        if offsets[start_index] is None or offsets[end_index] is None:
                            continue
                        # Skip answers with a length that is either < 0 or > max_answer_length
                        if (
                            end_index < start_index
                            or end_index - start_index + 1 > max_answer_length
                        ):
                            continue

                        answers.append(start_logit[start_index] + end_logit[end_index])

            
                if 1 < len(answers) < 200: # pad to same numbers of possible answers
                    zero_list = [0] * (200 - len(answers))
                    answers.extend(zero_list)
                elif len(answers) >= 200:
                    answers = answers[:200]

                probs[i][feature_index] += torch.tensor(softmax(answers))

    return probs

def get_batch_prob_dropout_split(dataloader, device, features, examples, n_drop=10):
    ## use tensor to save the answers
    c = 10
    model = AutoModelForQuestionAnswering.from_pretrained(strategy_model_dir).to(device)
    
    model.train()

    probs = torch.zeros([n_drop, len(dataloader.dataset), c])
    
    for i in range(n_drop):
        start_logits = []
        end_logits = []
        for batch in tqdm(dataloader, desc="Evaluating_prob_dropout"):
            batch = {key: value.to(device) for key, value in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)

            start_logits.append(outputs.start_logits.cpu().numpy())
            end_logits.append(outputs.end_logits.cpu().numpy())

        start_logits = np.concatenate(start_logits)
        end_logits = np.concatenate(end_logits)
        start_logits = start_logits[: len(features)]
        end_logits = end_logits[: len(features)]

        example_to_features = defaultdict(list)
        max_answer_length = 30
        n_best = 20
            
        for idx, feature in enumerate(features):
            example_to_features[feature["example_id"]].append(idx)

        n = 0
        for example in tqdm(examples, desc="Computing metrics"):
            if LOW_RES:
                example_id = example["qid"]
            else:
                example_id = example["id"]
            answers = []

            # Loop through all features associated with that example
            for feature_index in example_to_features[example_id]:
                start_logit = start_logits[feature_index]
                end_logit = end_logits[feature_index]
                offsets = features[feature_index]["offset_mapping"]

                start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
                end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
                for start_index in start_indexes:
                    for end_index in end_indexes:
                        # Skip answers that are not fully in the context
                        if offsets[start_index] is None or offsets[end_index] is None:
                            continue
                        # Skip answers with a length that is either < 0 or > max_answer_length
                        if (
                            end_index < start_index
                            or end_index - start_index + 1 > max_answer_length
                        ):
                            continue

                        answers.append(start_logit[start_index] + end_logit[end_index])

            
                if 1 < len(answers) < c: # pad to same numbers of possible answers
                    zero_list = [0] * (c - len(answers))
                    answers.extend(zero_list)
                elif len(answers) >= c:
                    answers = answers[:c]

                probs[i][feature_index] += torch.tensor(softmax(answers))

    return probs

query.py

In [244]:
from itertools import combinations_with_replacement

def class_combinations(c, n, m=np.inf):
    """ Generates an array of n-element combinations where each element is one of
    the c classes (an integer). If m is provided and m < n^c, then instead of all
    n^c combinations, m combinations aren randomly sampled.

    Arguments:
        c {int} -- the number of classes
        n {int} -- the number of elements in each combination

    Keyword Arguments:
        m {int} -- the number of desired combinations (default: {np.inf})

    Returns:
        np.ndarry -- An [m x n] or [n^c x n] array of integers in [0, c)
    """

    if m < c**n:
        # randomly sample combinations
        return np.random.randint(c, size=(int(m), n))
    else:
        p_c = combinations_with_replacement(np.arange(c), n)
        return np.array(list(iter(p_c)), dtype=int)

def H(x):
    """ Compute the element-wise entropy of x

    Arguments:
        x {torch.Tensor} -- array of probabilities in (0,1)

    Keyword Arguments:
        eps {float} -- prevent failure on x == 0

    Returns:
        torch.Tensor -- H(x)
    """
    return -(x)*torch.log(x)

def bald_query(n_pool, labeled_idxs, train_dataset, train_features, examples, device, n):
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    unlabeled_features = train_features.select(unlabeled_idxs)
    unlabeled_dataloader = DataLoader(
        unlabeled_data,
        collate_fn=default_data_collator,
        batch_size=MODEL_BATCH,
    )
    print('BALD querying starts!')
    print('Query {} data from {} unlabeled training data.'.format(n, len(unlabeled_data)))
    # to fasten development, turn dropout from 10 to 2
    probs = get_prob_dropout_split(unlabeled_dataloader, device, unlabeled_features, examples, n_drop=3)
    print('Got probability!')
    probs_mean = probs.mean(0)
    entropy1 = (-probs_mean*torch.log(probs_mean)).sum(1)
    entropy2 = (-probs*torch.log(probs)).sum(2).mean(0)
    uncertainties = entropy2 - entropy1
    # later on, we can use batch
    # uncertainties.sort()[1][:n] # select the best n # tensor([ 948, 1188, 1187, 1308, 1415,  534,  942,  918,  917, 1410,  913, 1372, ...
    return unlabeled_idxs[uncertainties.sort()[1][:n]]



### hide some steps for better view in development

Next we will preprocess the dataset (training and evaluation data).

In [8]:
train_dataset, train_features, val_dataset, val_features = preprocess_data(train_data, val_data)
# get the number of extra data after preprocessing
extra = len(train_dataset) - len(train_data)

Map:   0%|          | 0/1354 [00:00<?, ? examples/s]

Map:   0%|          | 0/1354 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Set seed and device

In [9]:
SEED = 1127
# os.environ['TORCH_HOME']='./basicmodel'
os.environ["CUDA_VISIBLE_DEVICES"] = str(2)

# fix random seed
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

all_acc = []
acq_time = []

Experiment Round 1

In [10]:
## record acc performance 
acc = np.zeros(ITERATION + 1) # quota/batch runs + iter_0
acc_em = np.zeros(ITERATION + 1)
acc = np.zeros(4) # quota/batch runs + iter_0
acc_em = np.zeros(4)

## generate initial labeled pool
n_pool = len(train_dataset)
print('n_pool:', n_pool)
labeled_idxs = np.zeros(n_pool, dtype=bool)

if LOW_RES:
    print('in low res setting')
    save_model(device, pretrain_model_dir, strategy_model_dir)
    ## print info
    print(DATA_NAME)
    print(STRATEGY_NAME)
else:
    print('not in low res setting')
    tmp_idxs = np.arange(n_pool)
    print('len(tmp_idxs):', len(tmp_idxs))
    np.random.shuffle(tmp_idxs)
    print('len(tmp_idxs):', len(tmp_idxs))
    
    if UNIQ_CONTEXT:
        print('in uc setting')
        tmp_idxs = tmp_idxs[:NUM_INIT_LB+extra]
        uc_tmp_idxs = get_unique_context(tmp_idxs, train_features, context_dict) # len() = almost num_query + extra
        print('len(uc_tmp_idxs):', len(uc_tmp_idxs))
        iter_0_labeled_idxs = get_unique_sample(labeled_idxs, uc_tmp_idxs, n_pool, train_features)
        c_id = get_final_c_id(iter_0_labeled_idxs, train_features) # len() = num_query
    else:
        print('not in uc setting')
        iter_0_labeled_idxs = get_unique_sample(labeled_idxs, tmp_idxs, n_pool, train_features)

    ## load the selected train data to DataLoader
    train_dataloader = DataLoader(
        train_dataset.select(indices=iter_0_labeled_idxs),
        shuffle=True,
        collate_fn=default_data_collator,
        batch_size=MODEL_BATCH,
    )

    num_update_steps_per_epoch = len(train_dataloader)
    num_training_steps = NUM_TRAIN_EPOCH * num_update_steps_per_epoch

    ## network
    model = AutoModelForQuestionAnswering.from_pretrained(get_model(MODEL_NAME)).to(device)
    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
    
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps,
    )
    ## print info
    print(DATA_NAME)
    print(STRATEGY_NAME)

    ## iteration 0 accuracy
    to_train(NUM_TRAIN_EPOCH, train_dataloader, device, model, optimizer, lr_scheduler)

## load the selected validation data to DataLoader
eval_dataloader = DataLoader(
    val_dataset, 
    collate_fn=default_data_collator, 
    batch_size=MODEL_BATCH
)

acc_scores_0 = get_pred(eval_dataloader, device, val_features, val_data) # add i=1 to use model from models_dir
acc[0] = acc_scores_0['f1']
acc_em[0] = acc_scores_0['exact_match']

print('Round 0\ntesting accuracy {}'.format(acc[0]))
print('testing accuracy em {}'.format(acc_em[0]))
# time = datetime.datetime.now()
# print('Time spent for init training:', (time - start))
print('\n')

n_pool: 1442
in low res setting
BioASQ
BatchBALDDropout


Evaluating_pred:   0%|          | 0/21 [00:00<?, ?it/s]

Computing metrics:   0%|          | 0/150 [00:00<?, ?it/s]

Round 0
testing accuracy 57.91499048455571
testing accuracy em 35.333333333333336




### end of better view

Iteration 1

In [ ]:
i = 1
total_query = NUM_QUERY + extra
		
## query
q_idxs = bald_query(n_pool, labeled_idxs, train_dataset, train_features, train_data, device, total_query)


In [56]:
q_idxs

array([ 873, 1408,  489, 1376,  925,  276,  789, 1312,  342,  931,  932,
         48,  352,  672, 1421,  593, 1401,  464, 1415, 1429, 1287, 1201,
       1200,  125,  861,  114,  594,  407,  533,  130, 1186,  540, 1438,
       1439,  939, 1431,  580,  589,  739,  738,  171,  609, 1106, 1116,
        886,  597,  465,  279, 1081, 1218,  290,  291,  595,  611,   29,
        539, 1083,  347,  630,  912,  785,  519,  813,  492, 1325,  966,
       1061, 1060,  990,  216,  217,  405,  467,  622,  962,  664, 1226,
        144, 1232,  365,  530, 1379, 1380, 1398, 1247,  913,  689,  339,
        353,   40,   55,  749,  967, 1123, 1434,  438,  559,  902,  262,
        627,  866,  100, 1114, 1113,  776,  422, 1172,  753,  303, 1251,
        750,   41,  450,   15,  596,  471,  314,   25,  356,  300,  321,
        127,  598, 1111,  416,  415, 1107, 1131, 1416,  147, 1290, 1427,
        196, 1313, 1222, 1067, 1068, 1066])

In [304]:
def batch_bald_query(n_pool, labeled_idxs, train_dataset, train_features, examples, device, n):
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    # unlabeled_features = train_features.select(unlabeled_idxs)
    print('BALD querying starts!')
    print('Query {} data from {} unlabeled training data.'.format(n, len(unlabeled_data)))
    m = 1e4  # number of MC samples for label combinations
    # if LOW_RES:
    #     num_sub_pool = len(unlabeled_data)
    # else:
    #     num_sub_pool = 500  # number of datapoints in the subpool from which we acquire
    num_sub_pool = 500  # number of datapoints in the subpool from which we acquire

    c = 10
    k = 10
    processing_batch_size = 8 # Model_batch=8 or 128

    # performing BatchBALD on the whole pool is very expensive, so we take a random subset of the pool.
    num_extra = len(unlabeled_data) - num_sub_pool

    if num_extra > 0:
        sub_pool_data, _ = torch.utils.data.random_split(unlabeled_data, [num_sub_pool, num_extra])
        sub_pool_idxs = np.array(sub_pool_data.indices)
    else:
        # even if we don't have enough data left to split, we still need to
        # call random_splot to avoid messing up the indexing later on
        sub_pool_data, _ = torch.utils.data.random_split(unlabeled_data, [len(unlabeled_data), 0])
        sub_pool_idxs = np.array(sub_pool_data.indices)

    # forward pass on the pool once to get class probabilities for each x
    pool_loader = torch.utils.data.DataLoader(sub_pool_data,
        batch_size=processing_batch_size, pin_memory=True, shuffle=False, collate_fn=default_data_collator)
    pool_features = train_features.select(sub_pool_idxs)

    pool_p_y = get_batch_prob_dropout_split(pool_loader, device, pool_features, examples, n_drop=k).permute(1, 0, 2) 
    # pool_p_y.shape = n * k * c = 500 * 3 * 200
    # this only need to be calculated once so we pull it out of the loop
    H2 = (H(pool_p_y).sum(axis=(1,2))/k)

    # get all class combinations
    c_1_to_n = class_combinations(k, n, m) # TODO: change from 'n' to 'total_query' # TODO: test1- change c to k
    # print('c_1_to_n.shape:', c_1_to_n.shape) # (10000, 138) ? # tensor of size [m * n]

    # tensor of size [m * k]
    p_y_1_to_n_minus_1 = None

    # store the indices of the chosen datapoints in the subpool
    best_sub_local_indices = []

    # create a mask to keep track of which indices we've chosen
    remaining_indices = torch.ones(len(sub_pool_data), dtype=bool)
    # remaining_indices = torch.ones(total_query, dtype=bool)
    for batch_n in range(n): # TODO: change from 'n' to 'total_query'
        # tensor of size [N * m * l] # [500, 10000, 3]
        p_y_n = pool_p_y[:, c_1_to_n[:, batch_n], :] # wierd here # TODO: test1
        # p_y_n = pool_p_y[:, :, c_1_to_n[:, batch_n]] # try with this
        # tensor of size [N * m * k]   
        if p_y_1_to_n_minus_1 == None:
            p_y_1_to_n = p_y_n
        # elif torch.tensor(0) in p_y_1_to_n_minus_1:
        #     p_y_1_to_n = p_y_n
        else:
            p_y_1_to_n = torch.einsum('mk,pmk->pmk', p_y_1_to_n_minus_1, p_y_n)

        # and compute the left entropy term
        H1 = H(p_y_1_to_n.mean(axis=2)).sum(axis=1)
        # if H1[0] == torch.tensor(0): break
        # scores is a vector of scores for each element in the pool.
        # mask by the remaining indices and find the highest scoring element
        scores = H1 - H2
        # uncertainties_ = H2 - H1
        
        # print(len(uncertainties_))
        best_local_index = torch.argmax(scores - 1*(~remaining_indices)).item()

        # if best_local_index == 0:
        #     print(f'Best idx {best_local_index} in {batch_n}')

        # print(f'Best idx {best_local_index}')
        # print(f'Best idxs {scores[remaining_indices].sort(descending=True)[1][0]}')
        
        best_sub_local_indices.append(best_local_index)
        # print(best_sub_local_indices)
        # save the computation for the next batch
        p_y_1_to_n_minus_1 = p_y_1_to_n[best_local_index]
        # print(p_y_1_to_n_minus_1[0][0])
        # remove the chosen element from the remaining indices mask
        remaining_indices[best_local_index] = False
        

    # we've subset-ed our dataset twice, so we need to go back through
    # subset indices twice to recover the global indices of the chosen data
    best_local_indices = np.array(sub_pool_idxs)[best_sub_local_indices]
    best_global_indices = np.array(unlabeled_idxs)[best_local_indices]
    assert len(set(best_global_indices)) == n, f'there are only {len(set(best_global_indices))} unique idx in q_idx'
    return best_global_indices

In [305]:
i = 1
total_query = NUM_QUERY + extra
		
q_idxs_bb = batch_bald_query(n_pool, labeled_idxs, train_dataset, train_features, train_data, device, total_query)

BALD querying starts!
Query 138 data from 1442 unlabeled training data.


Evaluating_prob_dropout:   0%|          | 0/63 [00:00<?, ?it/s]

Computing metrics:   0%|          | 0/1354 [00:00<?, ?it/s]

Evaluating_prob_dropout:   0%|          | 0/63 [00:00<?, ?it/s]

Computing metrics:   0%|          | 0/1354 [00:00<?, ?it/s]

Evaluating_prob_dropout:   0%|          | 0/63 [00:00<?, ?it/s]

Computing metrics:   0%|          | 0/1354 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [302]:
q_idxs_bb

array([ 173,  626,  194,  140, 1075,  176,  118, 1084,  918,  917,  137,
       1097,  232, 1168,  575, 1151,  496,  215,  786,  802,  619, 1331,
       1336, 1085,   92,  226,  828,  833, 1070,  187,  205,  959,  410,
       1155, 1401,  724,   60,  725,  682, 1228, 1234, 1110,   12,   87,
        434,  558,  663,  744,  941, 1235,  294,  384,  771,  789, 1159,
       1419,  453,   79,   88,   42,  871,  150,   77,   95,  699,  500,
        383, 1291, 1154, 1258,  938, 1345,  494,  411,  366, 1392,  210,
        333,  536,  456, 1082,  798,  250, 1022,  817,  632,  656, 1090,
       1093,  106,  681,  654,  914,  878,  671,  816,  385,   90,    4,
        700, 1409, 1284,  168,  653,  441,  449,  711, 1170,  664,   97,
       1043,  560,  354,  183,  259,  723,  722, 1079, 1096, 1014,  277,
        691,  762, 1176, 1058, 1037,  546,  315,  198, 1220,  238, 1039,
        462,  610,   85,  791,  690,  823])

In [364]:
unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
# unlabeled_features = train_features.select(unlabeled_idxs)
print('BALD querying starts!')
print('Query {} data from {} unlabeled training data.'.format(total_query, len(unlabeled_data)))
# TODO: change from 'n' to 'total_query'
m = 1e4  # number of MC samples for label combinations
# if LOW_RES:
#     num_sub_pool = len(unlabeled_data)
# else:
#     num_sub_pool = 500  # number of datapoints in the subpool from which we acquire
num_sub_pool = 500  # number of datapoints in the subpool from which we acquire

c = 10
k = 10
processing_batch_size = 128 # Model_batch=8 or 128

# performing BatchBALD on the whole pool is very expensive, so we take a random subset of the pool.
num_extra = len(unlabeled_data) - num_sub_pool

if num_extra > 0:
    sub_pool_data, _ = torch.utils.data.random_split(unlabeled_data, [num_sub_pool, num_extra])
    sub_pool_idxs = np.array(sub_pool_data.indices)
else:
    # even if we don't have enough data left to split, we still need to
    # call random_splot to avoid messing up the indexing later on
    sub_pool_data, _ = torch.utils.data.random_split(unlabeled_data, [len(unlabeled_data), 0])
    sub_pool_idxs = np.array(sub_pool_data.indices)

# forward pass on the pool once to get class probabilities for each x
pool_loader = torch.utils.data.DataLoader(sub_pool_data,
    batch_size=processing_batch_size, pin_memory=True, shuffle=False, collate_fn=default_data_collator)
pool_features = train_features.select(sub_pool_idxs)

pool_p_y = get_prob_dropout_split(pool_loader, device, pool_features, train_data, n_drop=k).permute(1, 0, 2) 
# pool_p_y.shape = n * k * c = 500 * 3 * 200
# TODO: change from 'examples' to 'train_data'
# this only need to be calculated once so we pull it out of the loop
H2 = (H(pool_p_y).sum(axis=(1,2))/k)

BALD querying starts!
Query 138 data from 1442 unlabeled training data.


Evaluating_prob_dropout:   0%|          | 0/4 [00:00<?, ?it/s]

Computing metrics:   0%|          | 0/1354 [00:00<?, ?it/s]

Evaluating_prob_dropout:   0%|          | 0/4 [00:00<?, ?it/s]

Computing metrics:   0%|          | 0/1354 [00:00<?, ?it/s]

...

Computing metrics:   0%|          | 0/1354 [00:00<?, ?it/s]

Evaluating_prob_dropout:   0%|          | 0/4 [00:00<?, ?it/s]

Computing metrics:   0%|          | 0/1354 [00:00<?, ?it/s]

Evaluating_prob_dropout:   0%|          | 0/4 [00:00<?, ?it/s]

Computing metrics:   0%|          | 0/1354 [00:00<?, ?it/s]

In [365]:
pool_p_y.shape

torch.Size([500, 20, 10])

In [368]:
c_1_to_n = class_combinations(c, total_query, m)
print('c_1_to_n.shape:', c_1_to_n[0])

c_1_to_n.shape: [5 9 9 9 3 4 5 4 8 4 0 7 2 4 4 8 8 3 3 4 5 5 9 1 7 7 5 2 3 6 5 0 5 3 5 6 8
 0 6 0 9 1 0 4 9 4 9 5 2 9 0 8 7 7 4 3 3 8 0 6 9 8 8 8 1 8 8 2 8 0 4 1 8 9
 1 3 5 5 4 6 0 1 9 1 2 0 8 0 8 4 8 2 8 3 1 2 5 3 8 4 9 3 5 5 0 1 6 9 8 7 1
 1 4 4 8 5 6 5 2 5 5 7 8 8 4 8 0 6 7 9 0 9 2 6 4 0 8 0]


In [378]:
# get all class combinations
c_1_to_n = class_combinations(c, total_query, m) # TODO: change from 'n' to 'total_query' # TODO: test1- change c to k
# print('c_1_to_n.shape:', c_1_to_n.shape) # (10000, 138) ? # tensor of size [m * n]

# tensor of size [m * k]
p_y_1_to_n_minus_1 = None

# store the indices of the chosen datapoints in the subpool
best_sub_local_indices = []

# create a mask to keep track of which indices we've chosen
remaining_indices = torch.ones(len(sub_pool_data), dtype=bool)
# remaining_indices = torch.ones(total_query, dtype=bool)
for batch_n in range(total_query): # TODO: change from 'n' to 'total_query'
    # tensor of size [N * m * l] # [500, 10000, 3]
    p_y_n = pool_p_y[:, c_1_to_n[:, batch_n], :] # wierd here, bc k was < c # TODO: test1
    # p_y_n = pool_p_y[:, :, c_1_to_n[:, batch_n]] # try with this
    # tensor of size [N * m * k]   
    if p_y_1_to_n_minus_1 == None:
        p_y_1_to_n = p_y_n
        # print(1)
    elif torch.tensor(0) in p_y_1_to_n_minus_1:
        p_y_1_to_n = p_y_n
        # print(2)
    else:
        p_y_1_to_n = torch.einsum('mk,pmk->pmk', p_y_1_to_n_minus_1, p_y_n)
        # print(3)

    # and compute the left entropy term
    H1 = H(p_y_1_to_n.mean(axis=2)).sum(axis=1)

    # if H1[0] == torch.tensor(0): break
    # scores is a vector of scores for each element in the pool.
    # mask by the remaining indices and find the highest scoring element
    scores = H1 - H2

    # uncertainties_ = H2 - H1

    remaining_scores = scores[remaining_indices]
    
    best_local_index = torch.argmax(scores - max(scores)*(~remaining_indices)).item()
    # best_local_index_ = remaining_scores.sort(descending=True)[1][0]

    # if remaining_indices[best_local_index] == False: break
    # if best_local_index == 0:
    #     print(f'Best idx {best_local_index} in {batch_n}')

    # print(f'Best idx {best_local_index}')
    # print(f'Best idxs {best_local_index_}')
    # print(f'2Best idxs {remaining_scores.sort(descending=True)[1][1]}')
    # print(f'3Best idxs {remaining_scores.sort(descending=True)[1][2]}')
    # print(remaining_scores[best_local_index_])
    # print(remaining_scores[remaining_scores.sort(descending=True)[1][1]])
    # print(remaining_scores[remaining_scores.sort(descending=True)[1][2]])
    # print(scores[best_local_index_])
    # print(scores[remaining_scores.sort(descending=True)[1][1]])
    # print(scores[remaining_scores.sort(descending=True)[1][2]])
    
    # best_sub_local_indices.append(best_local_index_)
    # # print(best_sub_local_indices)
    # # save the computation for the next batch
    # p_y_1_to_n_minus_1 = p_y_1_to_n[best_local_index_]
    # # print(p_y_1_to_n_minus_1[0][0])
    # # i = 1
    # # if remaining_indices[best_local_index_] == False:
    #     # print('H1:', H1[:5])

    #     # print(scores[:5])
    #     # print(len(remaining_scores))
    #     # print(remaining_scores.sort(descending=True)[0][:5])
    #     # print(remaining_scores.sort(descending=True)[1][:5])

    # # remove the chosen element from the remaining indices mask
    # remaining_indices[best_local_index_] = False

    best_sub_local_indices.append(best_local_index)
    # save the computation for the next batch
    p_y_1_to_n_minus_1 = p_y_1_to_n[best_local_index]
    # remove the chosen element from the remaining indices mask
    remaining_indices[best_local_index] = False
    

# we've subset-ed our dataset twice, so we need to go back through
# subset indices twice to recover the global indices of the chosen data
best_local_indices = np.array(sub_pool_idxs)[best_sub_local_indices]
best_global_indices = np.array(unlabeled_idxs)[best_local_indices]
assert len(set(best_global_indices)) == total_query, f'there are only {len(set(best_global_indices))} unique idx in q_idx'
# return best_global_indices

In [372]:
scores.index

AttributeError: 'Tensor' object has no attribute 'index'

In [373]:
max(scores)

tensor(1315.2615)

In [359]:
remaining_scores[311]

tensor(246.2740)

In [356]:
n = 0
for r in remaining_indices:
    if r == True:
        n += 1
n

383

In [379]:
best_global_indices

array([1078,  173,  118,  508, 1084,  211,  709,   85,  795,  796,  197,
        107, 1188, 1132,  218,  233,  417, 1097,  439,  137,  581,  141,
        199,  201,  549, 1034,  408, 1168, 1145, 1318,  800,  744, 1186,
       1092,  315,  271, 1044,  192,  457, 1030,  725,   95,  702, 1012,
       1009, 1185, 1054,  193, 1227,    4,  894,  893, 1205, 1155, 1013,
        426,  789,  154,  543, 1032,  971, 1058, 1064, 1314,  388,  852,
        939,  513,  985,  723, 1277,  722, 1062, 1344,  435,   64, 1035,
        706,  441,  724,  257,  812, 1079, 1373, 1275,  698, 1069, 1041,
        122,   79, 1025, 1419,  455, 1137,  526,  632, 1001,  144,  696,
        854,  873,  161,  663,  139, 1106, 1401, 1110,  682,  494,  628,
        354,  557,   77,  618,   63, 1171,  601, 1285, 1071,   44,  743,
       1015,  990, 1128, 1167,  701,  771,  649,  379,   54, 1376,   62,
        892, 1257,  382,  788,   56, 1424])

In [380]:
best_sub_local_indices

[321,
 77,
 434,
 378,
 146,
 47,
 289,
 53,
 153,
 307,
 494,
 91,
 331,
 185,
 282,
 456,
 388,
 43,
 368,
 60,
 243,
 391,
 181,
 475,
 24,
 387,
 472,
 353,
 112,
 418,
 403,
 367,
 354,
 57,
 54,
 208,
 186,
 80,
 267,
 467,
 55,
 180,
 199,
 263,
 269,
 428,
 215,
 462,
 258,
 424,
 9,
 140,
 265,
 29,
 187,
 249,
 122,
 189,
 228,
 393,
 392,
 370,
 155,
 435,
 276,
 90,
 319,
 309,
 312,
 222,
 492,
 236,
 426,
 247,
 72,
 16,
 464,
 330,
 463,
 279,
 96,
 419,
 232,
 219,
 364,
 375,
 133,
 266,
 87,
 495,
 158,
 344,
 8,
 455,
 485,
 323,
 410,
 217,
 152,
 313,
 334,
 79,
 245,
 56,
 459,
 246,
 422,
 135,
 115,
 221,
 99,
 188,
 211,
 179,
 318,
 62,
 343,
 175,
 49,
 465,
 498,
 397,
 411,
 200,
 229,
 61,
 166,
 117,
 310,
 70,
 288,
 105,
 399,
 453,
 385,
 93,
 425,
 438]

In [314]:
len(scores[remaining_indices])

478

In [242]:
assert 1 == total_query, f'there are only {len(set(best_global_indices))} unique idx in q_idx'

AssertionError: there are only 138 unique idx in q_idx

In [59]:
i = 1
total_query = NUM_QUERY + extra

unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
unlabeled_features = train_features.select(unlabeled_idxs)
unlabeled_dataloader = DataLoader(
    unlabeled_data,
    collate_fn=default_data_collator,
    batch_size=MODEL_BATCH,
)
print('BALD querying starts!')
print('Query {} data from {} unlabeled training data.'.format(total_query, len(unlabeled_data)))
# to fasten development, turn dropout from 10 to 2
probs = get_prob_dropout_split(unlabeled_dataloader, device, unlabeled_features, train_data, n_drop=3)

c = 3
k = 200
# print(probs.shape) # k n c
Y = probs.permute(1, 0, 2)
# print(torch_stack_probs.shape) # n k c
entropy1 = H(Y.mean(1)).sum(axis=1)
entropy2 = H(Y).sum(axis=(1,2))/k

uncertainties = entropy2 - entropy1 ### for getting the descending order
### same as -> return H1 - H2 from score(model, x, k=100)

unlabeled_idxs[uncertainties.sort()[1][:total_query]]

BALD querying starts!
Query 138 data from 1442 unlabeled training data.


Evaluating_prob_dropout:   0%|          | 0/181 [00:00<?, ?it/s]

Computing metrics:   0%|          | 0/1354 [00:00<?, ?it/s]

Evaluating_prob_dropout:   0%|          | 0/181 [00:00<?, ?it/s]

Computing metrics:   0%|          | 0/1354 [00:00<?, ?it/s]

Evaluating_prob_dropout:   0%|          | 0/181 [00:00<?, ?it/s]

Computing metrics:   0%|          | 0/1354 [00:00<?, ?it/s]

array([ 638, 1185, 1184,  225,  224, 1201, 1200,  746, 1202, 1402,  538,
        483, 1341, 1216,  412,  867, 1088, 1087,  357, 1010, 1009, 1008,
       1109, 1436, 1437,  648,  647,  527,  542,  707,  997,  998,    5,
        905, 1239, 1190,  180,  359,  358,  600, 1208,  924, 1271, 1129,
       1406,  980, 1192,    0, 1322, 1189,  629,  451,  293, 1246,  380,
        849, 1165,  156,  157,  376,  485,  484, 1181, 1432,  904, 1327,
        343, 1323, 1233, 1198, 1182,  987,  988, 1106, 1389,  132,  947,
       1111,  309, 1390, 1422,  890,  889,   37,   66, 1417,   32,  373,
        284,   40,  951,  922,  139,  645,  646,  573,  642,  862,  921,
       1211, 1397,  920,  166, 1433, 1197, 1196,  639,  289,  891,  954,
       1328,  124,  487,  223, 1410, 1317, 1126,  272, 1384,  949, 1241,
        946,   47,  935,    9, 1108,  636, 1019,  814,  304,  126,  590,
        452,   33, 1304, 1240,  530,  160])

In [63]:
uncertainties

tensor([-4.9013, -4.3057, -1.9604,  ..., -3.8115, -4.4537, -1.6220])